In [1]:
#Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import time
import json
from pprint import pprint
from scipy.stats import ttest_ind
from datetime import datetime
from config import yelp_key

In [2]:
county_datafile = "data/LOS_ANGELES_COUNTY_RESTAURANT_AND_MARKET_INSPECTIONS.csv"

county_data = pd.read_csv(county_datafile)


In [3]:
columns = ['ACTIVITY DATE','FACILITY ID','FACILITY NAME','PROGRAM STATUS','PE DESCRIPTION','FACILITY ADDRESS','FACILITY CITY','FACILITY ZIP','SCORE','GRADE']
cleaned_county_data = county_data[columns]
cleaned_county_data['ACTIVITY DATE'] = pd.to_datetime(cleaned_county_data['ACTIVITY DATE'])
cleaned_county_data.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,ACTIVITY DATE,FACILITY ID,FACILITY NAME,PROGRAM STATUS,PE DESCRIPTION,FACILITY ADDRESS,FACILITY CITY,FACILITY ZIP,SCORE,GRADE
0,2016-06-22,FA0179753,TEA BAR,ACTIVE,RESTAURANT (31-60) SEATS HIGH RISK,954 E ALOSTA AVE,AZUSA,91702,93,A
1,2016-09-07,FA0239319,SHEFA,ACTIVE,RESTAURANT (0-30) SEATS HIGH RISK,7275 MELROSE AVE,LOS ANGELES,90046,90,A
2,2016-09-09,FA0071154,SUBWAY,ACTIVE,RESTAURANT (0-30) SEATS MODERATE RISK,28788 THE OLD RD,VALENCIA,91355,98,A
3,2016-04-01,FA0144494,AFC SUSHI @ VONS #3138,ACTIVE,RESTAURANT (0-30) SEATS HIGH RISK,16550 SOLEDAD CANYON RD,CANYON COUNTRY,91387-3215,99,A
4,2016-04-01,FA0005693,ALBERTS MEXICAN FOOD,INACTIVE,RESTAURANT (0-30) SEATS HIGH RISK,2355 PECK RD,SOUTH EL MONTE,91733,92,A


In [4]:
cleaned_county_data.sort_values(by = ["ACTIVITY DATE","FACILITY ID","FACILITY ADDRESS"],inplace = True,ascending=True)
cleaned_county_data.count()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


ACTIVITY DATE       199950
FACILITY ID         199950
FACILITY NAME       199950
PROGRAM STATUS      199950
PE DESCRIPTION      199950
FACILITY ADDRESS    199950
FACILITY CITY       199950
FACILITY ZIP        199950
SCORE               199950
GRADE               199895
dtype: int64

In [5]:
unique_county_data = cleaned_county_data.drop_duplicates(subset = ['FACILITY ID'], keep = 'last')
#print(unique_county_data.count())
unique_county_data = unique_county_data.loc[(unique_county_data['PROGRAM STATUS'] != "INACTIVE"),:]
#print(unique_county_data.count())
unique_county_data = unique_county_data[unique_county_data['PE DESCRIPTION'].str.contains("RESTAURANT")]
#print(unique_county_data.count())
unique_county_data = unique_county_data[~unique_county_data['FACILITY NAME'].str.contains("#")]
#print(unique_county_data.count())
unique_county_data = unique_county_data.dropna(how='any')
print(unique_county_data.count())

ACTIVITY DATE       22512
FACILITY ID         22512
FACILITY NAME       22512
PROGRAM STATUS      22512
PE DESCRIPTION      22512
FACILITY ADDRESS    22512
FACILITY CITY       22512
FACILITY ZIP        22512
SCORE               22512
GRADE               22512
dtype: int64


In [44]:
sample_data = unique_county_data.sample(n=5000)
sample_data.to_csv("sample_data_ryan.csv")
sample_data

,ACTIVITY DATE,FACILITY ID,FACILITY NAME,PROGRAM STATUS,PE DESCRIPTION,FACILITY ADDRESS,FACILITY CITY,FACILITY ZIP,SCORE,GRADE
180976,2018-07-17,FA0166854,EL TEQUILENSE,ACTIVE,RESTAURANT (31-60) SEATS HIGH RISK,9126 TELEGRAPH RD,DOWNEY,90240,94,A
198269,2018-09-25,FA0058219,BERRYSTAR,ACTIVE,RESTAURANT (0-30) SEATS MODERATE RISK,39341 10TH ST W # C,PALMDALE,93551,92,A
197097,2018-09-19,FA0225142,SEAFOOD EXPRESS LA,ACTIVE,RESTAURANT (0-30) SEATS HIGH RISK,2122 BEVERLY BLVD,LOS ANGELES,90057,91,A
195507,2018-09-13,FA0028368,PHO ROWLAND,ACTIVE,RESTAURANT (31-60) SEATS HIGH RISK,18910 GALE AVE STE A,ROWLAND HEIGHTS,91748,93,A
188692,2018-08-15,FA0030460,JINO'S PARS,ACTIVE,RESTAURANT (61-150) SEATS HIGH RISK,5844 W MANCHESTER AVE,LOS ANGELES,90045,91,A
188742,2018-08-15,FA0023696,MARIE CALLENDER'S PIE SHOP,ACTIVE,RESTAURANT (61-150) SEATS HIGH RISK,7860 E FLORENCE AVE,DOWNEY,90241,94,A
190180,2018-08-22,FA0019252,DK DONER KING,ACTIVE,RESTAURANT (0-30) SEATS HIGH RISK,22349 SHERMAN WAY,CANOGA PARK,91303,95,A
158112,2018-04-11,FA0226479,C.C BBQ & SUSHI INC,ACTIVE,RESTAURANT (31-60) SEATS HIGH RISK,1006 E AVENUE J,LANCASTER,93535,98,A
195005,2018-09-12,FA0035895,LOS TRES HERMANOS,ACTIVE,RESTAURANT (31-60) SEATS HIGH RISK,7456 VAN NUYS BLVD,VAN NUYS,91405,90,A
178075,2018-06-30,FA0008815,BEL AIR BAY CLUB-UPPER CLUB,ACTIVE,RESTAURANT (151 + ) SEATS HIGH RISK,16801 PACIFIC COAST HWY,PACIFIC PALISADES,90272,93,A


In [ ]:
#headers = {"Authorization": "Bearer "+yelp_key}
#params = {"term": unique_county_data['FACILITY NAME'].iloc[0],"location": unique_county_data['FACILITY ZIP'].iloc[0]}
#target_url = "https://api.yelp.com/v3/businesses/search"

#yelp_reviews = requests.get(target_url, params, headers=headers).json()
#yelp_reviews

In [45]:
headers = {"Authorization": "Bearer "+ yelp_key}
target_url = "https://api.yelp.com/v3/businesses/search"

final_df = pd.DataFrame()
name = []
rating = []
review_count = []
price = []
category = []

#for index, row in sample_data.iloc[:10,:].iterrows():
for index, row in sample_data.iterrows():
    restr_name = row['FACILITY NAME']
    restr_zip = row['FACILITY ZIP']
    restr_addr = row['FACILITY ADDRESS']
    restr_city = row['FACILITY CITY']
    params = {"term": restr_name,"location": str(restr_addr + ", " + restr_city + ", "+restr_zip)}
    
    yelp_response = requests.get(target_url, params, headers=headers)
    
    if yelp_response.status_code != requests.codes.ok:
        print(yelp_response.text)
    else:
        yelp_data = yelp_response.json()
        try:
            name.append(yelp_data['businesses'][0]['name'])
            print(yelp_data['businesses'][0]['name'])
        except (KeyError, IndexError):
            name.append("NA")
            #print("Missing Name...")
        try:
            rating.append(yelp_data['businesses'][0]['rating'])
            #print(yelp_data['businesses'][0]['rating'])
        except (KeyError, IndexError):
            rating.append("NA")
            #print("Missing Rating...")
        try:
            review_count.append(yelp_data['businesses'][0]['review_count'])
            #print(yelp_data['businesses'][0]['review_count'])
        except (KeyError, IndexError):
            review_count.append("NA")
            #print("Missing Reviews...")    
        try:    
            price.append(yelp_data['businesses'][0]['price'])
            #print(yelp_data['businesses'][0]['price'])
        except (KeyError, IndexError):
            price.append("NA")
            #print("Missing Price...")
        try:    
            category.append(yelp_data['businesses'][0]['categories'][0]['title'])
            #print(yelp_data['businesses'][0]['categories'][0]['title'])
        except (KeyError, IndexError):
            category.append("NA")
            #print("Missing Category...")
    time.sleep(.1)


El Tequilense Restaurant
Berrystar Frozen Yogurt
Seafood Express
Pho Rowland Restaurant
Jino's Pars
Pie-Not The Aussie Style Bakery
Sushi Gen
Los Tres Hermanos
Bel-Air Bay Club
Birrieria La Barca Jalisco Restaurant
Rigo's Taco
Mr Churro
Edwards West Covina Stadium 18
2 For 1 Pizza
Loteria Grill
Panera Bread
Vivoli Cafe & Trattoria
BB's Cafe
Eagle Rock Noodle & Grill
Carrows Restaurants
California Wok
Sprouts Farmers Market
Crowne Plaza Los Angeles Harbor Hotel
Tierra Mia Coffee
The Coffee Bean & Tea Leaf
Rice Garden
J Trani's Ristorante
BrewCo
The Second City Hollywood
Los Cipotes salvodorian grill
Potato & Tea
Frutibionicos
A & T Burgers
Curry House CoCo Ichibanya
Asanebo
Trimana Grill
MulDaePo BBQ
China Great Buffet
Yummy.com
Carl's Jr.
Subway
Flame Broiler
Subway
Pane E Vino
Canton Kitchen
Cafe Cuba & Cakes
Polly's Pies Restaurant
Domino's Pizza
El Delfin
Okane Umi
Bettolino Kitchen
Summer Buffalo
Pho Oh! Ngon
East Garden Restaurant
Kogi Teriyaki
Fatburger
True Food Kitchen
Tavous K

Verde
The Little Izaka-Ya by Katsu-Ya Sherman Oaks
El Migueleno Restaurant
Champs Sports Pub
Tacos El Gavilan
Bristol Cafe
Nice Bakery
Eggs N Things
SunCafe Organic
Canoe House
Pollo a La Brasa Western
Royal Palace Banquet Hall
Oak Creek Golf Club
Santa Rita Jalisco Taco Truck
Villa Catrina
Bun Me!
Burgerim
Yo-Way
Cilantro Fresh Mexican Grill
Le Mervetty
Juan Pollo
Philippe the Original
Lenny's Deli
Robeks Juice
California Pizza Kitchen at Culver Center
101 Noodle Express
Kosuke
Wok Wok Express
Los Amigos Mexican Grill and Cantina
Yellow Basket
Premier Dessert Art
Auntie Anne's
SaMo Pizza
Marie Callender's
Class 302
Doreen's Tropical Fruit
Melody
Great Khan's Mongolian Grill
800 Degrees Neapolitan Pizzeria
Cali's Pizza
Bartha's Donuts
The Bazaar by José Andrés
Electric Owl
Taqueria of Hollywood
Javier's Century City
Pho VT
House Of Donuts
Christy's Donuts
La Reyna Tacos 2#
The Rice Street
SHUHARI Matcha Café
Colima Burgers
BJ's Restaurant & Brewhouse
Quiznos
Monja Taiker
Sham Tseng BBQ

Sunset Tower Hotel
Dan's Deli
Once Upon A Dream
Mandarin Chinese Food & Sushi
Napoli Pizza
L & H Cakes and Party Supplies
El Caney Market
Pina Pizza House
La Cabañita
Cabo Cantina
California Chicken Cafe
El Cholo
Tacos Reyes
Subway
Baja Beach Bar
Mr. Poké
The Donut House
Loaded Legz
The Original Rinaldi's Deli and Cafe
Las Lomas Restaurant
Nim Chan's Kitchen
ROK Sushi Kitchen
Subway
Bobaloca Manhattan Beach
Saigon Eden
MOD Pizza
Fresh Canes
Chop'd
Subway
Maestro Bakery & Café
Castaic Truck Stop
Thai Time Cuisine
Saraya Thai Cuisine
Barbarita Restaurant
Yei San Jib Korean Restaurant
The Craftsman Bar and Kitchen
The Flame Broiler
Daughter’s Deli
Yakuza Sushi
Westfield Santa Anita
Young King
Octopus Japanese Restaurant
Don Felix Restaurant
Lassens Natural Foods & Vitamins
Galbi King
Venice Beach Wines
Class 302
Subway
Five Star Chinese Food
Me Gusta Gourmet Tamales
New Pho 999
Q38 Noodle House
Taco Bell
Sushi Danku
Soundworks
Craigers Fudge, Ice Cream & Sweets
Tacos Chihuahua
Icho Izakay

Fuji Sushi & Rolls
Pupuseria Mi Salvador
Pizza Di Roma
Subway
Chipotle Mexican Grill
Pho Mely
The Los Angeles Banh Mi Company
Miceli's
Pamore Pizza
Noemi's Mexican Food
Daikokuya Little Tokyo
Dinah's Family Restaurant
Killer Shrimp
Casa Nostra Ristorante & Bar
Doubting Thomas
Tokyo Express
Paul's Kitchen
Uncle Reds Pizzeria
Rosewoods Family Restaurant
Estate Restaurant + Bar
Tonys Darts Away
San Pedro Fish Market and Restaurant
Tina's Tavern
India's Restaurant
Seven Grand
Thelma's Morning Cafe
Tacos Los Toritos
Frank's Restaurant
China Wok Express
Il Pastaio
Churroholic
The Counter
LA Crystal Hotel
ABC Seafood
Le Saigon
Crying Tiger
Super Pizza Veloz
Downey Pizza Company
Lebanon Shawarma
Sweet Lady Jane Bakery
Eagle Rock Brewery Public House
Grimaldi's Pizzeria
Nabee-Ya
Xi An Tasty
La Pergoletta
Douglas Burgers
Tip Top Hamburgers
Valencia Car Wash
Krispy Kreme
Panda King
Sol Maya
Wurstküche
El Burrito Jr
Bar 104
Subway
ABC Donuts
Tak's Coffee Shop
Super Donut House
Mulcaheys
Golden Bir

Antigua Bread
DeSano Pizza Bakery
Britannia Pub
Dunkin' Donuts
Tequilas Cantina & Grill
Ella's Jüs
Elysee Bakery & Cafe
Teriyakiya
El Ruizeñor Grill
La Casita Mexicana
Elbows Mac N' Cheese
Dunkin' Donuts
Guisados
Green Crush
Honey's Kettle Fried Chicken
Pagliacci's Trattoria Taverna
Pacific Dining Car
Spice Plus Restaurant & Grocery
Louis Burgers
McDonald's
Babytuna Sushi
Pho Kim
Shin Beijing
Four Seasons Hotel Los Angeles at Beverly Hills
Salvation Army Adult Rehabilitation Center
Boom Boom Chicken
Factory Tea Bar
Tutti Mangia Italian Grill
Restaurante Las Michoacanas
The Getty Center
Wow Bowl
Metro Ale House & Grill
Moderns Cafe
Sattdown Jamaican Grill
Trimana Express
Beijing Restaurant
Magee's Donuts
Abel's Bakery
El Pollo Loco
So Sushi
Thai Curry
Planet Pansit
The PCS Connection
RockSugar Southeast Asian Kitchen
Mercado Taqueria
Cafe Sprout
Street Churros
White Cup Donuts
New Starboard Attitude
WaBa Grill
Wokcano Asian Restaurant & Bar
Futaba Japanese Restaurant
La Sierra Nightclub

...

Rusty Pot Cafe
LJ's Partyland
Captain Lee's Seafood
Launica Mexican Food & Bakery
Jamba Juice
Second Round
Sharky's Woodfired Mexican Grill
Natalie Peruvian Seafood Restaurant
R Bar
The Rockefeller
Airport Royal Cuisine
Emy Burgers
Ozen Sushi
GUACS
Berry Bowl
El Zarape Melrose
Fonz's Restaurant
Sunny Grill
Slurpin' Ramen Bar
La Catedral Restaurant
China Wok Express
Fritto Misto
Angel Maid Bakery
Sushi 101
Nuevo San Miguel
La Vue
Del Campo Bakery
Sheraton Universal Hotel
Domino's Pizza
Brent's Delicatessen & Restaurant
The Health Winner at Westin Bonaventure
Fortune Cookie
Wildflour Bakery & Cafe
B-Bee Crepe and Boba
Boba Time
Vons
Heirloom Bakery
Bangkok Palace
El Taco Llama
Alpine Village Inn
The Buffalo Club
Panda Express
The Serving Spoon
Bravo Pizza & Chicken
Matt & Tony's
Hillstone
SD Coffee
Knockouts Gentlemen's Club
Salon Figueroa
Los Angeles Times Festival of Books
Dharma Vow Zen Sangha
Quality Food & Beverage
PizzaRossa
Buono's Authentic Pizzeria
Chinese Delight
Weiler's Origi

Urbane Cafe
Nairi Meat & Deli
Hae Jang Chon Korean BBQ Restaurant
Bagel Factory
Seven Grand
Pho Hana Vietnamese Cuisine
Fruiggies
Ruby BBQ Food
Gangnam Korean BBQ
California Science Center
A&W Seafood Restaurant
WaBa Grill
Kikka
Seven Grand
Dong Nae Gil
Half & Half Good Old Time
Valley Produce Market
Papa John's Pizza
G&K Burgers
Rainbow Donuts
Bud's Deli
Pizza Studio
Tower Cafe
Waterlily Cafe
Green Zone
Iroha Sushi of Tokyo
Pastries By Edie
The Yoga Collective
Lil' Simmzy's
Barone's
Nova's Grill
Miss Donuts and Bakery
Juice Paradise
Urth Caffé- Melrose
Fred 62
The Source Cafe
Halfway House Cafe
Deli Sis
Arto's Broadway Deli
Yalla Mediterranean
The Galley
Q Sushi
Vincenti Ristorante
La Tapatia Mexican Grill
Los Angeles Yacht Club
Toca Madera
Mimi's Cafe
Cafe Spot
My Father's BBQ
Cheng Du Restaurant
Wetzel's Pretzels
S & P Restaurant & Sports Bar
Jamba Juice
Sugar Suite
Urth Caffé - Santa Monica
Max's Mexican Cuisine
CoCo Fresh Tea & Juice
Cafe Sierra
Ramona's Mexican Food Products
Jona

Baby Bros Pizza & Wings
The Belmont
Blue Bottle Coffee
Kabab Way
Jersey Mike's Subs
Star Wok
Veracruz Mexican Restaurant
Hannosuke
Go Greek Yogurt
The Oyster Bar SKC
Cafe X2O, Glendora
Antonio's Tacos & Kabob
Tickle Tree Cafe
The Coffee Bean & Tea Leaf
Spitz - Little Tokyo
Texas Donuts
Sasoun Bakery
Restaurant & Pupuseria El Ranchon
WaBa Grill
ISA Japanese Restaurant
Sun Ha Jang
Marston's
Masala Grill
Randy's Donuts
Sweet Lab
Denny's
Slater's 50/50 - Pasadena
Julie's Pizzeria
Twist Eatery
Bubble Republic
Guisados
Denny's
Park Twin Theatres
Pho 24
Wing N Stuff
Spudnuts Donuts
The Edison
Roxbury Cafe
Village Pizza
Foothill Cinema 10
Bakery House
Shin-Sen-Gumi Yakitori -Gardena
Balboa Sub Shop
Oasis Pupuseria
Hudson House
Señor G's Fresh & Healthy Mexican Food
Chipotle Mexican Grill
Top Donuts & Sandwiches
Millennium Biltmore Los Angeles
El Pollo Loco
Brazuca
Tommy's Famous Drive-Thru
Koutoubia
Jimmy John's
Tamon Sushi
Boulevard3
Beech Street
Jitlada
Corner Place
Flower Gallery
Luchini Pi

La Esperanza Bakery
Maral's Pastry
Art's Table
California Ramen Factory
Ritter's Steam Kettle Cooking
ArcLight Cinemas
Pa Ord Noodle
Vanda's Thai Kitchen
Bea's Bakery
The Cheesecake Factory
Da Vinci Ice Cream
LA Decks
Antelope Valley Country Club
The India Restaurant
Kang Ho Dong Baekjeong
Java Man
Fernando's Taco Inn
Afterburn Fitness
Michell's Donut House
Jasmine House Chinese Cuisine
Quiet Cannon Conference & Event Center
New Hong Kong Chinese Express
Mi Lindo Cuscatlan Restaurant y Pupuseria
Subway
King's Fish House - Calabasas
Taco Nazo
Guelaguetza Restaurante
You & Me Dining
Mr. Masala
Wetzel's Pretzels
Pinkberry
India's Oven
Pat's II Cocktail Lounge
Red Seven
Whole Foods Market
Fina Estampa
Thomas Hamburgers
Frugos Yogurt
The Pizzeria Factory
Seventeen Cafe
IHOP
Chillin' Thai Cuisine
The 1776
Max's Mexican Cuisine
Zensen Sushi Express
Cafe Rio Mexican Grill
Eggslut
El Sabroso
Byby's Bakery
BMC PHO Banh Mi Che Cali
Tel Aviv Grill
Durango's Restaurant
Shogun Japanese Restaurant
Sk

Tsujita LA Artisan Noodle
Get Bbul
Fonda Don Chon
Thai Kitchen
The Salvation Army
Cup My Cakez
Rafo's Cafe
Ackee Bamboo Jamaican Cuisine
Masis Bakery
Petty Cash Taqueria
Hound Dog Hot Dog Shop
Arbat Banquet Hall
Thirty5 Degrees
Rock & Brews - El Segundo
Donut Galore
GRANVILLE
Baskin-Robbins
La Fiesta Brava
Waldorf Astoria Beverly Hills
Tatang
Wingstop
Vespaio
Big Mama's & Papa's Pizzeria
Birdies
Gondola Pizza
Dorado Bakery
Pacific Dining Car
Lucero Bakery
Ragin Cajun Cafe
Dave & Buster's
SUSHIRAW
Liaison Restaurant + Lounge
Cafe Allegro
O Sushi
China Express
Chili's
Crescent Hotel Beverly Hills
Flaming Wok
GuYi Restaurant
Beer Belly
Mariscos Los Arcos
La Farfala Coffee Shop & Deli
Clifton's Republic
Denny's
Tropicana Bakery & Cuban Café
Crepes de Paris
Subway
Twinkle Brown Sugar
Hong Kong Express
Feu Pho Kitchen
American Airlines Admirals Club
Panda Garden
Bottega Louie
Mango Tango Juicebar & Grill
Lawry's Carvery
Blazing Stone Pizza
Sushi Planet Woodland Hills
Blaze Fast-Fire'd Pizza


In [46]:
final_df['Name'] = name
final_df['Inspection_Score'] = sample_data['SCORE'].tolist()
final_df['Inspection_Grade'] = sample_data['GRADE'].tolist()
final_df['Rating'] = rating
final_df['Reviews'] = review_count
final_df['Price'] = price
final_df['Category'] = category

final_df

,Name,Inspection_Score,Inspection_Grade,Rating,Reviews,Price,Category
0,El Tequilense Restaurant,94,A,4,42,$,Mexican
1,Berrystar Frozen Yogurt,92,A,4.5,10,$,Desserts
2,Seafood Express,91,A,4,266,$,Seafood
3,Pho Rowland Restaurant,93,A,4,408,$,Vietnamese
4,Jino's Pars,91,A,4,477,$$,Persian/Iranian
5,Pie-Not The Aussie Style Bakery,94,A,4,826,$,Bakeries
6,NA,95,A,NA,NA,NA,NA
7,Sushi Gen,98,A,4.5,3536,$$$,Sushi Bars
8,Los Tres Hermanos,90,A,3.5,362,$$,Mexican
9,Bel-Air Bay Club,93,A,4.5,78,NA,Social Clubs


In [47]:
final_df['Reviews'] = pd.to_numeric(final_df['Reviews'],errors = 'coerce').fillna(0).astype(np.int64)
q1 = np.percentile(final_df['Reviews'],25)
q1

41.0

In [48]:
percentiled_df = final_df.loc[final_df['Reviews'] > q1, :]
percentiled_df

,Name,Inspection_Score,Inspection_Grade,Rating,Reviews,Price,Category
0,El Tequilense Restaurant,94,A,4,42,$,Mexican
2,Seafood Express,91,A,4,266,$,Seafood
3,Pho Rowland Restaurant,93,A,4,408,$,Vietnamese
4,Jino's Pars,91,A,4,477,$$,Persian/Iranian
5,Pie-Not The Aussie Style Bakery,94,A,4,826,$,Bakeries
7,Sushi Gen,98,A,4.5,3536,$$$,Sushi Bars
8,Los Tres Hermanos,90,A,3.5,362,$$,Mexican
9,Bel-Air Bay Club,93,A,4.5,78,NA,Social Clubs
10,Birrieria La Barca Jalisco Restaurant,97,A,3.5,236,$$,Mexican
12,Rigo's Taco,97,A,3.5,42,$,Mexican


In [52]:
percentiled_df.Inspection_Grade.value_counts()

A    3544
B     176
C      13
Name: Inspection_Grade, dtype: int64

In [83]:
cleaned_percentiled_data = percentiled_df.drop_duplicates(subset = ['Name', 'Reviews'], keep = 'first')
cleaned_percentiled_data2 = cleaned_percentiled_data.reset_index(drop=True)

cleaned_percentiled_data3 = cleaned_percentiled_data2.drop(cleaned_percentiled_data2.index[219])
cleaned_percentiled_data3.head()


,Name,Inspection_Score,Inspection_Grade,Rating,Reviews,Price,Category
0,El Tequilense Restaurant,94,A,4,42,$,Mexican
1,Seafood Express,91,A,4,266,$,Seafood
2,Pho Rowland Restaurant,93,A,4,408,$,Vietnamese
3,Jino's Pars,91,A,4,477,$$,Persian/Iranian
4,Pie-Not The Aussie Style Bakery,94,A,4,826,$,Bakeries


In [81]:
cleaned_percentiled_data4 = cleaned_percentiled_data3.reset_index(drop=True)
cleaned_percentiled_data4.head(220)

,Name,Inspection_Score,Inspection_Grade,Rating,Reviews,Price,Category
0,El Tequilense Restaurant,94,A,4,42,$,Mexican
1,Seafood Express,91,A,4,266,$,Seafood
2,Pho Rowland Restaurant,93,A,4,408,$,Vietnamese
3,Jino's Pars,91,A,4,477,$$,Persian/Iranian
4,Pie-Not The Aussie Style Bakery,94,A,4,826,$,Bakeries
5,Sushi Gen,98,A,4.5,3536,$$$,Sushi Bars
6,Los Tres Hermanos,90,A,3.5,362,$$,Mexican
7,Bel-Air Bay Club,93,A,4.5,78,NA,Social Clubs
8,Birrieria La Barca Jalisco Restaurant,97,A,3.5,236,$$,Mexican
9,Rigo's Taco,97,A,3.5,42,$,Mexican


In [82]:
# Save Data to csv
cleaned_percentiled_data4.to_csv("final_output_ryan.csv")